In [1]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to audio descriptions collection

In [ ]:
audio_descriptions = database['modified_audio_descriptions_x']     # last updated audio descriptions collection

#### Drop audio descriptions with no audio clip info in audio clips collection

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'modified_audio_clips',    # last updated audio clips collection
            'localField': 'audio_clips', 
            'foreignField': '_id', 
            'as': 'ad_ac'
        }
    }, {
        '$addFields': {
            'audio_clips': '$ad_ac._id'
        }
    }, {
        '$project': {
            'ad_ac': 0
        }
    }, {
        '$out': 'modified_audio_descriptions_xx'   # new audio descriptions collection 
    }
]

audio_descriptions.aggregate(pipeline)